In [ ]:
import os
import math
import random
from typing import Dict, List, Tuple

import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt

from skimage.measure import regionprops, label as sk_label
from skimage.morphology import remove_small_objects, binary_opening, binary_closing, disk
from skimage.segmentation import slic
from skimage.util import img_as_float

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score

from xgboost import XGBClassifier

from IPython.display import display
from PIL import Image
from ipywidgets import Dropdown, Text, Button, VBox, Output
from pathlib import Path
import joblib


RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

DATA_ROOT = r"C:\BPA\data"


CLASS_NAMES = ["gunshot", "impact"]
LABEL_TO_INT = {c: i for i, c in enumerate(CLASS_NAMES)}
INT_TO_LABEL = {i: c for c, i in LABEL_TO_INT.items()}

print("DATA_ROOT =", DATA_ROOT)



In [ ]:
# Cell 2 — Building image index from DATA_ROOT 

import glob
from pathlib import Path

# Extensions we consider as images
IMG_EXTS = ("*.jpg", "*.jpeg", "*.png", "*.bmp", "*.tif", "*.tiff")


def infer_label_from_filename(path: str) -> Tuple[str, int]:
    """

    Assumption given:
      - Files starting with 'GS-' (case-insensitive) => gunshot
      - Everything else => impact

    """
    name = os.path.basename(path)
    upper = name.upper()

    if upper.startswith("GS-"):
        label_str = "gunshot"
    else:
        label_str = "impact"

    label_int = LABEL_TO_INT[label_str]
    return label_str, label_int


def build_image_index(data_root: str) -> pd.DataFrame:
    """
    Scans DATA_ROOT recursively and creates a DataFrame with:
      pattern_id, filepath, label, label_int

    pattern_id is taken from the filename without extension
      (GS-1.jpg -> 'GS-1')
    """
    rows = []
    root = Path(data_root)

    if not root.is_dir():
        raise RuntimeError(f"DATA_ROOT does not exist: {data_root}")

    for ext in IMG_EXTS:
        pattern = os.path.join(str(root), "**", ext)
        files = glob.glob(pattern, recursive=True)
        for f in files:
            pid = os.path.splitext(os.path.basename(f))[0]
            label_str, label_int = infer_label_from_filename(f)

            rows.append(
                {
                    "pattern_id": pid,
                    "filepath": f,
                    "label": label_str,
                    "label_int": label_int,
                }
            )

    df = pd.DataFrame(rows)
    if df.empty:
        raise RuntimeError("No images found")

    # Optional: sort for nice, stable ordering
    df = df.sort_values(by=["label", "pattern_id"]).reset_index(drop=True)
    return df



images_df = build_image_index(DATA_ROOT)

print("Image index built from DATA_ROOT:")
print(f"  Total images found: {len(images_df)}\n")

print("  Images per class (inferred):")
print(images_df["label"].value_counts())

display(images_df.head())


In [ ]:
# Cell 3 — Image loading & visualization 

from PIL import Image, ImageFile
import numpy as np

def load_image_bgr(path: str) -> np.ndarray:
 
    pil_img = Image.open(path).convert("RGB")
    img_rgb = np.array(pil_img)    # full resolution
    img_bgr = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR)
    return img_bgr


#Show a safely resized preview for visualization ONLY.
def show_safe_preview(path: str, max_dim: int = 1800):

    pil_img = Image.open(path).convert("RGB")

    w, h = pil_img.size
    scale = max_dim / max(w, h)
    if scale < 1.0:
        new_w = int(w * scale)
        new_h = int(h * scale)
        pil_small = pil_img.resize((new_w, new_h))
    else:
        pil_small = pil_img

    plt.figure(figsize=(10, 10))
    plt.imshow(pil_small)
    plt.axis("off")
    plt.title(Path(path).name)
    plt.show()


test_row = images_df.iloc[0]
show_safe_preview(test_row["filepath"])


In [ ]:
# Cell 4 — Simple grayscale-based mask (for contingency seeds)

def segment_gray_baseline(img_bgr: np.ndarray,
                          min_area: int = 30,
                          scale_factor: float = 1.0) -> np.ndarray:

    h, w = img_bgr.shape[:2]

    # Optionally downscale for speed 
    if scale_factor != 1.0:
        new_w = int(w * scale_factor)
        new_h = int(h * scale_factor)
        img_proc = cv2.resize(
            img_bgr, (new_w, new_h),
            interpolation=cv2.INTER_AREA if scale_factor < 1.0 else cv2.INTER_LINEAR
        )
    else:
        img_proc = img_bgr

    gray = cv2.cvtColor(img_proc, cv2.COLOR_BGR2GRAY)
    inv = 255 - gray

    # Otsu threshold
    _, thr = cv2.threshold(inv, 0, 255,
                           cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    mask_bool = thr.astype(bool)
    mask_bool = remove_small_objects(mask_bool, min_size=min_area)
    mask_small = (mask_bool.astype(np.uint8) * 255)

    # Upscale mask back to original resolution
    if scale_factor != 1.0:
        mask = cv2.resize(mask_small, (w, h), interpolation=cv2.INTER_NEAREST)
        mask = (mask > 0).astype(np.uint8) * 255
    else:
        mask = mask_small

    return mask



In [ ]:
# Cell 5 - Multi pass HSV based blood segmentation

import matplotlib.pyplot as plt

def segment_blood_hsv(
    img_bgr: np.ndarray,
    base_min_size: int = 5,
    debug_show: bool = False,
) -> Tuple[np.ndarray, np.ndarray]:
 

    # Preprocess
    img_blur = cv2.GaussianBlur(img_bgr, (3, 3), 0)
    hsv = cv2.cvtColor(img_blur, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)

    # CLAHE on V channel
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    v_eq = clahe.apply(v)
    hsv_eq = cv2.merge([h, s, v_eq])

    #Red hue mask 
    lower_red1 = np.array([0, 30, 10], dtype=np.uint8)
    upper_red1 = np.array([12, 255, 255], dtype=np.uint8)
    lower_red2 = np.array([160, 30, 10], dtype=np.uint8)
    upper_red2 = np.array([180, 255, 255], dtype=np.uint8)

    mask1 = cv2.inRange(hsv_eq, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv_eq, lower_red2, upper_red2)
    mask_red = cv2.bitwise_or(mask1, mask2)

    if mask_red.sum() == 0:
        empty = np.zeros(mask_red.shape, dtype=np.uint8)
        labels_empty = sk_label(empty.astype(bool), connectivity=2)
        if debug_show:
            show_image_and_mask(img_bgr, empty, title="No red detected")
        return empty, labels_empty

    # stats over red pixels
    s_red = s[mask_red > 0]
    v_red = v_eq[mask_red > 0]
    if len(s_red) < 50:
        s_red = s.flatten()
        v_red = v_eq.flatten()

    H, W = img_bgr.shape[:2]
    area = H * W

    def run_pass(s_thresh, v_thresh, min_size):
        sat_ok = (s >= s_thresh).astype(np.uint8) * 255
        val_ok = (v_eq >= v_thresh).astype(np.uint8) * 255
        m = cv2.bitwise_and(mask_red, sat_ok)
        m = cv2.bitwise_and(m, val_ok)

        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
        m = cv2.morphologyEx(m, cv2.MORPH_OPEN, kernel, iterations=1)
        m = cv2.morphologyEx(m, cv2.MORPH_CLOSE, kernel, iterations=2)

        mb = m.astype(bool)
        mb = remove_small_objects(mb, min_size=min_size)
        return mb

    # Percentile thresholds
    p_s50 = np.percentile(s_red, 50)
    p_s30 = np.percentile(s_red, 30)
    p_s15 = np.percentile(s_red, 15)

    p_v50 = np.percentile(v_red, 50)
    p_v30 = np.percentile(v_red, 30)
    p_v15 = np.percentile(v_red, 15)

    strict_s = max(30, p_s50)
    strict_v = max(25, p_v50)
    min_size_strict = max(base_min_size * 4, int(0.000001 * area))

    mid_s = max(20, p_s30)
    mid_v = max(15, p_v30)
    min_size_mid = max(base_min_size * 2, int(0.0000007 * area))

    relax_s = max(10, p_s15)
    relax_v = max(8,  p_v15)
    min_size_relax = max(base_min_size, int(0.0000003 * area))

    # Masks
    mb_strict = run_pass(strict_s, strict_v, min_size_strict)
    mb_mid    = run_pass(mid_s,    mid_v,    min_size_mid)
    mb_relax  = run_pass(relax_s,  relax_v,  min_size_relax)

    # Combine (union)
    mb_combined = mb_strict | mb_mid | mb_relax

    mb_final = remove_small_objects(mb_combined, min_size=base_min_size)
    mask_out = mb_final.astype(np.uint8) * 255
    labels   = sk_label(mb_final, connectivity=2)

    if debug_show:
        preview = cv2.resize(img_bgr, (1000, 1000 * img_bgr.shape[0] // img_bgr.shape[1]))

        strict_mask = mb_strict.astype(np.uint8) * 255
        mid_mask    = mb_mid.astype(np.uint8) * 255
        relax_mask  = mb_relax.astype(np.uint8) * 255

        plt.figure(figsize=(16, 10))

        plt.subplot(2, 3, 1)
        plt.imshow(cv2.cvtColor(preview, cv2.COLOR_BGR2RGB))
        plt.title("Preview")
        plt.axis("off")

        plt.subplot(2, 3, 2)
        plt.imshow(mask_out, cmap="gray")
        plt.title(f"Final mask (stains: {labels.max()})")
        plt.axis("off")

        plt.subplot(2, 3, 4)
        plt.imshow(strict_mask, cmap="gray")
        plt.title("Strict")
        plt.axis("off")

        plt.subplot(2, 3, 5)
        plt.imshow(mid_mask, cmap="gray")
        plt.title("Mid")
        plt.axis("off")

        plt.subplot(2, 3, 6)
        plt.imshow(relax_mask, cmap="gray")
        plt.title("Relaxed")
        plt.axis("off")

        plt.tight_layout()
        plt.show()

    return mask_out, labels



test_row = images_df.iloc[0]
test_img = load_image_bgr(test_row["filepath"])  

mask_hsv, labels_hsv = segment_blood_hsv(test_img, debug_show=True)

#Result
show_safe_preview(test_row["filepath"])
print("Pattern:", test_row["pattern_id"], "| Label:", test_row["label"])
print("Stains detected (Variant B):", labels_hsv.max())


In [ ]:
# Cell 6 — Stain-level feature extraction

def extract_stain_features_variantB(
    img_bgr: np.ndarray,
    labels: np.ndarray,
    pattern_id: str
) -> pd.DataFrame:

    hsv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)

    props = regionprops(labels, intensity_image=v)

    if len(props) == 0:
        return pd.DataFrame()

    # First pass: get pattern centroid (area-weighted)
    centroids = []
    areas = []
    for reg in props:
        if reg.area < 5:
            continue
        centroids.append(reg.centroid)  # (row, col)
        areas.append(reg.area)

    if not centroids:
        return pd.DataFrame()

    centroids = np.array(centroids)
    areas = np.array(areas)
    pattern_cy = (centroids[:, 0] * areas).sum() / areas.sum()
    pattern_cx = (centroids[:, 1] * areas).sum() / areas.sum()

    rows = []

    for reg in props:
        if reg.area < 5:
            continue

        major = reg.major_axis_length or 0.0
        minor = reg.minor_axis_length or 0.0
        if major <= 0 or minor <= 0:
            continue

        # Shape filters (similar spirit to paper)
        if reg.eccentricity <= 0.3:
            continue

        aspect = minor / major
        if aspect <= 0:
            continue
        if aspect < (np.pi / 18.0):
            continue

        if reg.solidity < 0.75:
            continue

        filled_area = reg.filled_area
        ellipse_area = (np.pi * major * minor) / 4.0
        if filled_area <= 0 or ellipse_area <= 0:
            continue
        if (minor ** 2 / filled_area) > 1:
            continue
        if (reg.area / filled_area) < 0.95:
            continue

        cy, cx = reg.centroid
        dx = cx - pattern_cx
        dy = cy - pattern_cy
        dist_to_centroid = math.sqrt(dx * dx + dy * dy)

        major_adj = major * (reg.area / ellipse_area)
        impact_angle = aspect  # like paper's minor/major
        adjusted_impact_angle = minor / major_adj if major_adj > 0 else np.nan

        # Colour stats inside stain
        region_mask = (labels == reg.label)
        h_vals = h[region_mask]
        s_vals = s[region_mask]
        v_vals = v[region_mask]

        rows.append({
            "pattern_id": pattern_id,
            "stain_id": reg.label,
            "area": reg.area,
            "major_axis": major,
            "minor_axis": minor,
            "eccentricity": reg.eccentricity,
            "solidity": reg.solidity,
            "centroid_x": float(cx),
            "centroid_y": float(cy),
            "orientation": float(reg.orientation),  # radians
            "impact_angle": float(impact_angle),
            "adjusted_impact_angle": float(adjusted_impact_angle),
            "distance_to_centroid": float(dist_to_centroid),
            "mean_intensity": float(reg.mean_intensity),
            "intensity_stddev": float(np.std(reg.intensity_image[reg.image])),
            "hue_mean": float(np.mean(h_vals)),
            "hue_std": float(np.std(h_vals)),
            "sat_mean": float(np.mean(s_vals)),
            "sat_std": float(np.std(s_vals)),
            "val_mean": float(np.mean(v_vals)),
            "val_std": float(np.std(v_vals)),
            "shade": float(np.mean(v_vals)),
            "evenness": float(np.std(v_vals)),
        })

    return pd.DataFrame(rows)

stains_test = extract_stain_features_variantB(test_img, labels_hsv, test_row["pattern_id"])
print("Stain table for first image:")
stains_test.head()


In [ ]:
# Cell 7 — Pattern level global + local features

def circular_variance_radians(angles: np.ndarray) -> float:
    if len(angles) == 0:
        return np.nan
    s = np.sin(angles).mean()
    c = np.cos(angles).mean()
    R = math.sqrt(s * s + c * c)
    return 1.0 - R

def compute_global_features_variantB(df: pd.DataFrame) -> Dict[str, float]:
    cols = [
        "area", "major_axis", "minor_axis", "solidity",
        "impact_angle", "adjusted_impact_angle", "distance_to_centroid",
        "mean_intensity", "intensity_stddev",
        "hue_mean", "hue_std", "sat_mean", "sat_std",
        "val_mean", "val_std", "shade", "evenness"
    ]
    out = {}
    n = len(df)

    out["num_stains"] = float(n)

    for c in cols:
        if n and c in df.columns:
            out[f"{c}_mean"] = float(df[c].mean())
            out[f"{c}_std"]  = float(df[c].std())
        else:
            out[f"{c}_mean"] = 0.0
            out[f"{c}_std"]  = 0.0

    if n and "orientation" in df.columns:
        out["orientation_cvar"] = float(
            circular_variance_radians(df["orientation"].to_numpy())
        )
    else:
        out["orientation_cvar"] = 0.0

    return out


def compute_local_features(df: pd.DataFrame,
                           n_rings: int = 40,
                           n_vbins: int = 40) -> Dict[str, float]:
    out = {}
    if len(df) == 0:
        out["fract_stains_ring_15_25"] = 0.0
        out["fract_stains_ring_25_35"] = 0.0
        out["ring_with_max_stains"] = -1
        out["fract_stains_vbin_25_35"] = 0.0
        out["vbin_with_max_stains"] = -1
        return out

    # radial rings based on distance
    d = df["distance_to_centroid"].to_numpy()
    if np.nanmax(d) == 0:
        d_norm = np.zeros_like(d)
    else:
        d_norm = d / np.nanmax(d)

    ring_idx = np.floor(d_norm * n_rings).astype(int)
    ring_idx = np.clip(ring_idx, 0, n_rings - 1)
    ring_counts = np.bincount(ring_idx, minlength=n_rings)
    total = ring_counts.sum()
    if total == 0:
        total = 1

    r15_25 = ring_counts[15:25].sum()
    r25_35 = ring_counts[25:35].sum()

    out["fract_stains_ring_15_25"] = r15_25 / total
    out["fract_stains_ring_25_35"] = r25_35 / total
    out["ring_with_max_stains"] = int(np.argmax(ring_counts))

    # vertical bins based on centroid
    y = df["centroid_y"].to_numpy()
    if np.nanmax(y) == np.nanmin(y):
        y_norm = np.zeros_like(y)
    else:
        y_norm = (y - np.nanmin(y)) / (np.nanmax(y) - np.nanmin(y))

    vbin_idx = np.floor(y_norm * n_vbins).astype(int)
    vbin_idx = np.clip(vbin_idx, 0, n_vbins - 1)
    v_counts = np.bincount(vbin_idx, minlength=n_vbins)
    v_total = v_counts.sum()
    if v_total == 0:
        v_total = 1

    v25_35 = v_counts[25:35].sum()

    out["fract_stains_vbin_25_35"] = v25_35 / v_total
    out["vbin_with_max_stains"] = int(np.argmax(v_counts))

    return out


def pattern_features_variantB(img_bgr: np.ndarray,
                              pattern_id: str) -> Dict[str, float]:

    mask, labels = segment_blood_hsv(img_bgr)
    stains_df = extract_stain_features_variantB(img_bgr, labels, pattern_id)

    global_feats = compute_global_features_variantB(stains_df)
    local_feats = compute_local_features(stains_df)

    feat = {**global_feats, **local_feats}
    feat["pattern_id"] = pattern_id
    feat["valid_stains"] = float(len(stains_df))
    return feat




In [ ]:
# Cell 8 — Build images_df from the WHOLE DATA_ROOT using filename prefixes

import os
import glob
import pandas as pd

IMG_EXTS = ("*.jpg", "*.jpeg", "*.png", "*.bmp", "*.tif", "*.tiff")

def infer_label_from_filename(filename: str):

    name = os.path.basename(filename)
    upper = name.upper()

    if upper.startswith("GS-"):
        return "gunshot", 0
    else:
        return "impact", 1


def collect_images_from_root(root: str) -> pd.DataFrame:
    records = []
    # recursive=True picks up subfolders like 'Gunshot'
    for ext in IMG_EXTS:
        pattern = os.path.join(root, "**", ext)
        files = glob.glob(pattern, recursive=True)
        for f in files:
            label_str, label_int = infer_label_from_filename(f)
            pid = os.path.splitext(os.path.basename(f))[0]
            records.append(
                {
                    "pattern_id": pid,
                    "filepath": f,
                    "label": label_str,
                    "label_int": label_int,
                }
            )
    df = pd.DataFrame(records)
    if df.empty:
        raise RuntimeError(
            f"No images found in {root}. Check DATA_ROOT or extensions."
        )
    return df

images_df = collect_images_from_root(DATA_ROOT)

print("Total images found in DATA_ROOT (all folders):", len(images_df))
print("By class (inferred from filename):")
print(images_df["label"].value_counts())
images_df.head()


In [ ]:
# Cell 9 — FULL SIZE dataset with every image

def build_dataset_variantB_full(images_df: pd.DataFrame) -> Tuple[pd.DataFrame, np.ndarray]:
    rows = []
    y_list = []
    
    total = len(images_df)
    print("\n Building Dataset")
    print(f"Total images to process: {total}\n")

    for idx, row in images_df.iterrows():
        pid = str(row["pattern_id"])
        path = row["filepath"]
        label_int = int(row["label_int"])

        print(f"[{idx+1}/{total}] {pid} | {path}")

       
        try:
            img = load_image_bgr(path)        
            orig_h, orig_w = img.shape[:2]
        except Exception as e:
            print(f" [LOAD ERROR] Could not load: {path}")
            print(f"  Reason: {e}\n")
            continue

        # Feature extraction
        try:
            feats = pattern_features_variantB(img, pid)

            # save size columns
            feats["orig_h"] = orig_h
            feats["orig_w"] = orig_w
            feats["resized_h"] = orig_h
            feats["resized_w"] = orig_w

            rows.append(feats)
            y_list.append(label_int)

            print(f"Features extracted.\n")

        except Exception as e:
            print(f"[FEATURE ERROR] pattern_id={pid}")
            print(f"Reason: {e}\n")
            continue

    #Build DataFrame
    if not rows:
        raise RuntimeError("No images processed successfully. Check errors")

    X_df = pd.DataFrame(rows)
    if "pattern_id" in X_df.columns:
        X_df = X_df.set_index("pattern_id")

    y = np.array(y_list, dtype=int)

  
    print("\n FULL-SIZE Dataset Build Complete")
    print(f"Total images in dataset: {total}")
    print(f"Successfully processed:  {len(X_df)}")
    print(f"Failed images:           {total - len(X_df)}")

    if len(X_df) != total:
        print("\nSome images had errors. Scroll above for detailed logs.")



    return X_df, y


X_B, y_B = build_dataset_variantB_full(images_df)
print("Dataset shape:", X_B.shape, "labels:", y_B.shape)
X_B.head()


In [ ]:
#Saving as CSV
X_B_csv = X_B.copy()
X_B_csv["label_int"] = y_B

csv_path = "features.csv"
X_B_csv.to_csv(csv_path, index=True)

print(f"Saved features + size info to: {csv_path}")
display(X_B_csv.head())


In [ ]:
#Path to the CSV 
CSV_PATH = r"C:\BPA\features.csv" 
df = pd.read_csv(CSV_PATH)
print("Columns:", df.columns.tolist())
print(df.head())

In [ ]:
# ONLY GS- is gunshot. Everything else is impact.
if "pattern_id" not in df.columns:
    if df.index.name is not None:
        df = df.reset_index().rename(columns={df.index.name: "pattern_id"})
    else:
        raise RuntimeError("No 'pattern_id' column or index in CSV – please check the file.")

def infer_label_from_pattern_id(pid: str):

    name = str(pid).upper()
    if name.startswith("GS-"):
        return "gunshot", 0
    else:
        return "impact", 1

labels_str = []
labels_int = []

for pid in df["pattern_id"]:
    s, i = infer_label_from_pattern_id(pid)
    labels_str.append(s)
    labels_int.append(i)

df["label"] = labels_str
df["label_int"] = labels_int

LABEL_TO_INT = {"gunshot": 0, "impact": 1}
INT_TO_LABEL = {v: k for k, v in LABEL_TO_INT.items()}

print("Label distribution (recomputed from GS rule):")
unique, counts = np.unique(df["label_int"].values, return_counts=True)
for u, c in zip(unique, counts):
    print(f"  int {u} -> {INT_TO_LABEL[u]:8s} : {c} samples")

# Non-feature columns to exclude
NON_FEATURE_COLS = {"pattern_id", "label", "label_int"}

feature_cols = [c for c in df.columns if c not in NON_FEATURE_COLS]

X_B = df[feature_cols].copy()
y_B = df["label_int"].astype(int).values

print("X_B shape:", X_B.shape)
print("y_B shape:", y_B.shape)


In [ ]:
#XG Boost
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

RANDOM_SEED = 42

pos_label = LABEL_TO_INT["gunshot"]
n_pos = np.sum(y_B == pos_label)
n_neg = np.sum(y_B != pos_label)
print(f"Gunshot count: {n_pos}, Impact count: {n_neg}")

scale_pos_weight = n_neg / max(1, n_pos)
print("scale_pos_weight:", scale_pos_weight)

X_train, X_test, y_train, y_test = train_test_split(
    X_B.values.astype(float),
    y_B,
    test_size=0.25,
    random_state=RANDOM_SEED,
    stratify=y_B,
)

xgb_model = XGBClassifier(
    n_estimators=500,
    max_depth=4,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.9,
    random_state=RANDOM_SEED,
    n_jobs=-1,
    eval_metric="logloss",
    scale_pos_weight=scale_pos_weight,  
)

xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)  

print("\nXGBoost classification report:")
print(classification_report(
    y_test,
    y_pred,
    target_names=[INT_TO_LABEL[0], INT_TO_LABEL[1]],
))


In [ ]:
#Random forest check 
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(
    n_estimators=500,
    max_depth=None,
    random_state=RANDOM_SEED,
    n_jobs=-1,
    class_weight="balanced",  
)

rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

print("\nRandomForest classification report (balanced):")
print(classification_report(
    y_test,
    y_pred_rf,
    target_names=[INT_TO_LABEL[0], INT_TO_LABEL[1]],
))


In [ ]:
# Train on full dataset
xgb_model_final = XGBClassifier(
    n_estimators=500,
    max_depth=4,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.9,
    random_state=RANDOM_SEED,
    n_jobs=-1,
    eval_metric="logloss",
    scale_pos_weight=scale_pos_weight,
)

xgb_model_final.fit(X_B.values.astype(float), y_B)

MODEL_DIR = Path(r"C:\BPA\model")
MODEL_DIR.mkdir(parents=True, exist_ok=True)
MODEL_PATH = MODEL_DIR / "variantB_xgb.joblib"

joblib.dump(
    {
        "model": xgb_model_final,
        "feature_cols": feature_cols,
        "label_to_int": LABEL_TO_INT,
        "int_to_label": INT_TO_LABEL,
    },
    MODEL_PATH,
)

print("Saved model to:", MODEL_PATH)


In [ ]:
# Cell 13 — Contingency: superpixel-based mask refinement

def refine_mask_with_superpixels(img_bgr: np.ndarray,
                                 seed_mask: np.ndarray,
                                 n_segments: int = 400,
                                 compactness: float = 10.0,
                                 prob_thresh: float = 0.5) -> np.ndarray:
    """
    Contingency segmentation:
      - Use seed_mask (from grayscale baseline) as weak labels.
      - Run SLIC superpixels.
      - Train RF on superpixel-level features 
      - Predict stain probability per superpixel.
      - Threshold + morphology → refined mask.
    """
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    img_float = img_as_float(img_rgb)

    sp_labels = slic(
        img_float,
        n_segments=n_segments,
        compactness=compactness,
        start_label=0
    )

    hsv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)

    seed_bool = seed_mask.astype(bool)
    H, W = seed_mask.shape
    yy, xx = np.indices(seed_mask.shape)

    feat_list = []
    seed_labels = []

    for lab_id in np.unique(sp_labels):
        mask_sp = (sp_labels == lab_id)
        if mask_sp.sum() == 0:
            continue

        h_vals = h[mask_sp]
        s_vals = s[mask_sp]
        v_vals = v[mask_sp]

        h_mean = float(np.mean(h_vals))
        s_mean = float(np.mean(s_vals))
        v_mean = float(np.mean(v_vals))

        x_norm = float(xx[mask_sp].mean() / W)
        y_norm = float(yy[mask_sp].mean() / H)

        pos_frac = seed_bool[mask_sp].mean()
        if pos_frac >= 0.8:
            label_sp = 1
        elif pos_frac <= 0.05:
            label_sp = 0
        else:
            label_sp = -1  # unlabeled

        feat_list.append([h_mean, s_mean, v_mean, x_norm, y_norm])
        seed_labels.append(label_sp)

    feats = np.array(feat_list, dtype=float)
    seed_labels = np.array(seed_labels, dtype=int)

    mask_labeled = seed_labels >= 0
    if mask_labeled.sum() < 5:
        # not enough seeds; just return the seed mask
        return seed_mask

    X_train = feats[mask_labeled]
    y_train = seed_labels[mask_labeled]

    clf = RandomForestClassifier(
        n_estimators=100,
        random_state=RANDOM_SEED,
        n_jobs=-1
    )
    clf.fit(X_train, y_train)

    probs = clf.predict_proba(feats)[:, 1]

    prob_map = np.zeros_like(seed_mask, dtype=float)
    idx = 0
    for lab_id in np.unique(sp_labels):
        mask_sp = (sp_labels == lab_id)
        prob_map[mask_sp] = probs[idx]
        idx += 1

    refined_bool = prob_map >= prob_thresh
    refined_bool = binary_opening(refined_bool, disk(1))
    refined_bool = binary_closing(refined_bool, disk(2))

    refined_mask = refined_bool.astype(np.uint8) * 255
    return refined_mask

print("Contingency refinement function ready")


In [ ]:
# Cell 14- Load trained Variant B model     

import pandas as pd
import joblib
from pathlib import Path

MODEL_PATH = Path(r"C:\BPA\model\variantB_xgb.joblib")  

bundle = joblib.load(MODEL_PATH)
use_model = bundle["model"]
use_feat_cols = bundle["feature_cols"]
LABEL_TO_INT = bundle["label_to_int"]
INT_TO_LABEL = bundle["int_to_label"]

print(f"Loaded trained Variant B model bundle from: {MODEL_PATH}")
print("LABEL_TO_INT:", LABEL_TO_INT)
print("INT_TO_LABEL:", INT_TO_LABEL)
print(f"Number of features: {len(use_feat_cols)}")

CONF_THRESHOLD = 0.75
print(f"Contingency will trigger if max probability < {CONF_THRESHOLD}")


In [ ]:
# Cell 15 - Single image prediction using Variant B + contingency

import numpy as np

def predict_with_model(
    img_path: str,
    model,
    feature_cols,
    conf_threshold: float = CONF_THRESHOLD
):

    
    img_bgr = load_image_bgr(img_path)
    pattern_id = Path(img_path).stem

    #features from variant B 
    X_row_B, mask_B = features_for_inference_variantB(
        img_bgr=img_bgr,
        pattern_id=pattern_id,
        feature_cols=feature_cols
    )

    # prediction
    probs = model.predict_proba(X_row_B.values.astype(float))[0]
    max_idx = int(np.argmax(probs))
    max_prob = float(probs[max_idx])
    main_label = INT_TO_LABEL[max_idx]   

    print(f"[Variant B] {pattern_id} → {main_label} (conf = {max_prob:.3f})")

    if max_prob >= conf_threshold:
        return main_label, max_prob, dict(zip(INT_TO_LABEL.values(), probs)), {
            "used_contingency": False,
            "mask_main": mask_B,
            "mask_contingency": None,
        }

    # contigency
    print(f"Low confidence (< {conf_threshold}), running contingency...")

    seed_mask = segment_gray_baseline(img_bgr, min_area=30, scale_factor=1.0)
    refined_mask = refine_mask_with_superpixels(img_bgr, seed_mask)

    labels_refined = sk_label(refined_mask > 0)
    stains_df = extract_stain_features_variantB(img_bgr, labels_refined, pattern_id)

    gl = compute_global_features_variantB(stains_df)
    loc = compute_local_features(stains_df)
    gl["valid_stains"] = len(stains_df)
    feat_all_C = {**gl, **loc}

    row_C = {c: 0.0 for c in feature_cols}
    for k, v in feat_all_C.items():
        if k in row_C:
            row_C[k] = float(v)

    X_row_C = pd.DataFrame([row_C], columns=feature_cols)
    probs_C = model.predict_proba(X_row_C.values.astype(float))[0]

    max_idx_C = int(np.argmax(probs_C))
    max_prob_C = float(probs_C[max_idx_C])
    cont_label = INT_TO_LABEL[max_idx_C]   

    print(f"[Contingency] {pattern_id} → {cont_label} (conf = {max_prob_C:.3f})")

    return cont_label, max_prob_C, dict(zip(INT_TO_LABEL.values(), probs_C)), {
        "used_contingency": True,
        "mask_main": mask_B,
        "mask_contingency": refined_mask,
    }


In [ ]:
# Cell 16 - Variant B inference feature builder

from typing import List, Tuple
import pandas as pd
import numpy as np

def features_for_inference_variantB(
    img_bgr: np.ndarray,
    pattern_id: str,
    feature_cols: List[str]
) -> Tuple[pd.DataFrame, np.ndarray]:
 
    #Segmentation of blood in HSV 
    mask_B, labels_B = segment_blood_hsv(img_bgr)

    #Extracting stain level features
    stains_df = extract_stain_features_variantB(img_bgr, labels_B, pattern_id)

    #Aggregating to global + local pattern features
    gl = compute_global_features_variantB(stains_df)
    loc = compute_local_features(stains_df)
    gl["valid_stains"] = len(stains_df)

    feat_all = {**gl, **loc}

    # Align to training feature columns (filling missing values with 0.0)
    row = {c: 0.0 for c in feature_cols}
    for k, v in feat_all.items():
        if k in row:
            row[k] = float(v)

    X_row = pd.DataFrame([row], columns=feature_cols)

    return X_row, mask_B


In [ ]:
# Cell 17 - Test on one image

test_path = r"C:\BPA\Test2.jpg"  

label, conf, prob_dict, extra = predict_with_model(
    img_path=test_path,
    model=use_model,
    feature_cols=use_feat_cols,
    conf_threshold=CONF_THRESHOLD,
)

print("\nDecision:")
print(f"  Label      : {label}")
print(f"  Confidence : {conf:.3f}")
print(f"  Used contingency {extra['used_contingency']}")
